### splitting the 80k images into real and fake directories based on the csv labels

In [ ]:
import os
import pandas as pd
import shutil
from tqdm import tqdm


# --- CONFIGURATION ---
base_path = '../80k'
csv_path = os.path.join(base_path, 'train.csv')

# destination directories
train_dir = os.path.join(base_path, 'organized_train')
os.makedirs(os.path.join(train_dir, 'real'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'fake'), exist_ok=True)

# csv loading
df = pd.read_csv(csv_path)

print(f"🚀 Répartition de 80k images en cours...")

success = 0
errors = 0

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    # filename train_data/nom_image.jpg"
    relative_path = row['file_name'] 
    label = int(row['label'])
    
    # full path to the source image
    src_path = os.path.join(base_path, relative_path)
    
    
    pure_filename = os.path.basename(relative_path)
    
    if label == 1:
        dst_path = os.path.join(train_dir, 'fake', pure_filename)
    else:
        dst_path = os.path.join(train_dir, 'real', pure_filename)
    
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)
        success += 1
    else:
        
        if errors == 0:
            print(f"\n❌ Premier échec sur : {src_path}")
        errors += 1

print(f"\n✅ Terminé !")
print(f"📊 Déplacées : {success} | Introuvables : {errors}")

🚀 Répartition de 80k images en cours...


100%|██████████| 79950/79950 [00:06<00:00, 13236.01it/s]


✅ Terminé !
📊 Déplacées : 79950 | Introuvables : 0


### count images with width and height <= 256px 

In [6]:
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS = None 

DATA_ROOT = "../80k"
TARGET_MIN = 256

def check_dataset():
    paths = {
        "Train Fake": os.path.join(DATA_ROOT, 'train/fake'),
        "Train Real": os.path.join(DATA_ROOT, 'train/real'),
    }

    print(f"🚀 Analyse des images (Seuil min: {TARGET_MIN}x{TARGET_MIN}px)...")
    
    global_stats = {}

    for label, folder in paths.items():
        count_valid = 0
        count_total = 0
        
        if not os.path.exists(folder):
            print(f"⚠️ Dossier non trouvé : {folder}")
            continue

        files = [f for f in os.listdir(folder) if f.lower().endswith(('png', 'jpg', 'jpeg', 'webp'))]
        
        for filename in files:
            count_total += 1
            file_path = os.path.join(folder, filename)
            try:
                with Image.open(file_path) as img:
                    w, h = img.size
                    if w >= TARGET_MIN and h >= TARGET_MIN:
                        count_valid += 1
            except:
                continue # Image corrompue ou illisible
        
        global_stats[label] = (count_valid, count_total)
        print(f"✅ {label} : {count_valid} valides sur {count_total}")

    print("\n--- BILAN FINAL ---")
    total_v = sum(v[0] for v in global_stats.values())
    print(f"Total d'images exploitables pour l'entraînement : {total_v}")

check_dataset()

🚀 Analyse des images (Seuil min: 256x256px)...
✅ Train Fake : 39763 valides sur 39975
✅ Train Real : 39763 valides sur 39975

--- BILAN FINAL ---
Total d'images exploitables pour l'entraînement : 79526


### performing center crop + resize en place on big images

In [7]:
import os
from PIL import Image
from tqdm import tqdm


Image.MAX_IMAGE_PIXELS = None 

DATA_ROOT = "../80k"
TARGET_SIZE = (256, 256)

def preprocess_on_place():
    folders = [
        os.path.join(DATA_ROOT, 'train/fake'),
        os.path.join(DATA_ROOT, 'train/real'),
    ]

    for folder in folders:
        if not os.path.exists(folder):
            continue
            
        print(f"\n📦 Traitement de : {folder}")
        files = [f for f in os.listdir(folder) if f.lower().endswith(('png', 'jpg', 'jpeg', 'webp'))]
        
        for filename in tqdm(files):
            file_path = os.path.join(folder, filename)
            try:
                with Image.open(file_path) as img:
                    # 1. Conversion en RGB (essentiel pour les JPG et le CNN)
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    
                    # 2. Calcul du Center Crop
                    w, h = img.size
                    min_dim = min(w, h)
                    left = (w - min_dim) / 2
                    top = (h - min_dim) / 2
                    right = (w + min_dim) / 2
                    bottom = (h + min_dim) / 2
                    
                    img = img.crop((left, top, right, bottom))
                    
                    # 3. Redimensionnement final
                    img = img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)
                    
                    # 4. Sauvegarde (on remplace l'original par un JPG propre)
                    # On change l'extension en .jpg si nécessaire pour l'uniformité
                    base = os.path.splitext(file_path)[0]
                    new_path = base + ".jpg"
                    
                    img.save(new_path, "JPEG", quality=90)
                    
                    # Si l'ancien fichier avait une extension différente (ex: .png), on le supprime
                    if file_path != new_path:
                        os.remove(file_path)
                        
            except Exception as e:
                print(f"❌ Erreur sur {filename}: {e}")

if __name__ == "__main__":
    confirm = input("⚠️ Ce script va modifier DEFINITIVEMENT tes images en 256x256. Tape 'OK' pour continuer : ")
    if confirm.upper() == "OK":
        preprocess_on_place()
        print("\n✨ Preprocessing terminé ! Ton dataset est prêt pour le CNN.")


📦 Traitement de : ../80k/train/fake


100%|██████████| 39975/39975 [03:10<00:00, 210.08it/s]



📦 Traitement de : ../80k/train/real


100%|██████████| 39975/39975 [03:07<00:00, 213.34it/s]


✨ Preprocessing terminé ! Ton dataset est prêt pour le CNN.


### Train Val Test Split

In [9]:
import os
import shutil
import random
from tqdm import tqdm

def split_dataset(source_base, output_base, split_ratio=(0.8, 0.1, 0.1)):
    categories = ['real', 'fake']
    phases = ['train', 'val', 'test']
    
    for cat in categories:
        src_dir = os.path.join(source_base, cat)
        files = os.listdir(src_dir)
        random.shuffle(files)
        
        # Calcul des index
        n = len(files)
        train_end = int(n * split_ratio[0])
        val_end = train_end + int(n * split_ratio[1])
        
        file_split = {
            'train': files[:train_end],
            'val': files[train_end:val_end],
            'test': files[val_end:]
        }
        
        for phase in phases:
            dest_dir = os.path.join(output_base, phase, cat)
            os.makedirs(dest_dir, exist_ok=True)
            
            for f in tqdm(file_split[phase], desc=f"📦 {phase}/{cat}"):
                shutil.move(os.path.join(src_dir, f), os.path.join(dest_dir, f))

# --- CONFIGURATION ---
# Dossier où tu as tes 80k organisés
new_data_path = '../80k/train' 
# Dossier final qui recevra tout
final_path = './DATASET_V3'

split_dataset(new_data_path, final_path)
print("\n✅ Split terminé ! Tu peux maintenant y ajouter tes anciennes 48k images.")

📦 test/fake: 100%|██████████| 3998/3998 [00:00<00:00, 50453.96it/s]


✅ Split terminé ! Tu peux maintenant y ajouter tes anciennes 48k images.


### Adding the new images (80k) to the images/ folder

In [10]:
import os
import shutil
from tqdm import tqdm

# --- CONFIGURATION ---
source_base = './DATASET_V3'
target_base = '../images'

phases = ['train', 'val', 'test']
categories = ['real', 'fake']

print("🚀 Début du transfert avec préfixe 'new_'...")

for phase in phases:
    for cat in categories:
        src_dir = os.path.join(source_base, phase, cat)
        dst_dir = os.path.join(target_base, phase, cat)
        
        # Création du dossier de destination s'il n'existe pas
        os.makedirs(dst_dir, exist_ok=True)
        
        if not os.path.exists(src_dir):
            continue
            
        files = os.listdir(src_dir)
        for f in tqdm(files, desc=f"Moving {phase}/{cat}"):
            src_path = os.path.join(src_dir, f)
            
            # Nouveau nom avec préfixe
            new_name = f"new_{f}"
            dst_path = os.path.join(dst_dir, new_name)
            
            # Déplacement (shutil.move est plus rapide que copy)
            shutil.move(src_path, dst_path)

print("\n✅ Transfert terminé ! Tes images sont maintenant dans ../images/")

🚀 Début du transfert avec préfixe 'new_'...


Moving test/fake: 100%|██████████| 3998/3998 [00:00<00:00, 46657.71it/s]


✅ Transfert terminé ! Tes images sont maintenant dans ../images/


---

### Final Count

In [11]:
#printing the number of images in each folder after the move
print("\n📊 Répartition finale dans ../images :")
for phase in ['train', 'val', 'test']:
    for cat in ['real', 'fake']:
        folder = os.path.join('../images', phase, cat)
        count = len(os.listdir(folder)) if os.path.exists(folder) else 0
        print(f"📁 {phase}/{cat}: {count} images")


📊 Répartition finale dans ../images :
📁 train/real: 55973 images
📁 train/fake: 55978 images
📁 val/real: 6995 images
📁 val/fake: 6997 images
📁 test/real: 6997 images
📁 test/fake: 6998 images
